# Cleaning Data From PubMed API

In [26]:
import pandas as pd
import numpy as np
import os

import warnings
warnings.filterwarnings('ignore')

#### Pull In Scraped Data & Combine into One DataFrame

In [27]:
raw_files = os.listdir('../data/raw_pubmed/')

In [28]:
try:
    raw_files.remove('.ipynb_checkpoints')
except:
    pass

In [29]:
#set the order of columns
cols = ['pmid','title','text','date', 'citations']

In [30]:
# create a dataframe of each csv and store in a list
df_list = []
for file in raw_files: 
    df = pd.read_csv(f'../data/raw_pubmed/{file}')
    df = df[cols]
    df_list.append(df)

In [31]:
#concatenate all of the dataframes in the list into one large dataframe
pm_df = pd.DataFrame()
for df in df_list:
    pm_df = pd.concat([pm_df, df],ignore_index=True)

pm_df.shape

(3151, 5)

In [32]:
pm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3151 entries, 0 to 3150
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pmid       3151 non-null   int64  
 1   title      3151 non-null   object 
 2   text       3151 non-null   object 
 3   date       3128 non-null   object 
 4   citations  2500 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 123.2+ KB


In [33]:
#replace null citations with 0 (that's what null represents in this case)
pm_df['citations'].fillna(0, inplace=True)

In [34]:
#convert num of citations to int
pm_df['citations'] = pm_df['citations'].apply(int)

In [35]:
#drop missing date observations
pm_df.dropna(inplace=True)

#### Clean Messy Date Column

In [36]:
from datetime import datetime

In [37]:
pm_df['date']

0       2020 May 16;395(10236):1579-1586.
1                2020 May 13;20(10):2774.
2             Jul-Aug 2020;14(4):569-573.
3                       2020 May 12;6:16.
4                  2020;26(29):3569-3578.
                      ...                
3146                2019 Apr 20;91:72-78.
3147                2018 Jul;116:286-299.
3148        2018 Apr 27;122(9):1290-1301.
3149           2019 Mar 25;20(2):682-689.
3150              2018 May;19(7):629-650.
Name: date, Length: 3128, dtype: object

This column is incredibly messy. I only need a rough estimate to be able to account for any effects of the time that an article has been live, so I will try to brute-force extract a month and year from each cell. 

In [38]:
# function to extract the publication month and year from the date string 
def extract_date(datestring):
    
    #the months appear to be listed as "Jan", "Feb", etc.
    months_dict = {'Jan' : 1, 'Feb' : 2,'Mar' : 3,'Apr' : 4,
         'May' : 5,'Jun' : 6,'Jul' : 7,'Aug' : 8,
         'Sep' : 9, 'Oct' : 10,'Nov' : 11, 'Dec' : 12}
    
    #set up a list of all years that were scraped
    years = list(range(2010,2021))
    
    #set up empty strings to store the months and years 
    pub_month = ''
    pub_year = ''

    #look for months and years in the datestring cell
    for month in months_dict.keys():
        if month in datestring:
            pub_month = str(months_dict[month]).zfill(2)
        else:
            pass
    for year in years:
        if str(year) in datestring:
            pub_year = str(year)
        else:
            pass
    
    #if we can't find the publication year, we have to drop this one- could be off by a lot
    #if we can't find the month, will pick a month in the middle; so the maximum it could
    #be off by is six months. not perfect, but it's a decision. 
    
    if pub_year == '': 
        return None
    elif pub_month == '':
        pub_month = '6'
    else: 
        pass
    
    return pub_year+'-'+pub_month   

In [39]:
# apply date extraction function
pm_df['date'] = pm_df['date'].apply(extract_date)

In [40]:
#we don't have a simple way to get publication days for each of these, so will make all the first of the monm
pm_df['date'] = pd.to_datetime(pm_df['date'],format='%Y-%m')

In [41]:
pm_df.head()

,pmid,title,text,date,citations
0,32416782,Artificial intelligence and the future of glob...,Concurrent advances in information technology ...,2020-05-01,10
1,32414188,Future Is Unlicensed: Private 5G Unlicensed Ne...,This paper aims to unlock the unlicensed band ...,2020-05-01,0
2,32413821,A review of modern technologies for tackling C...,Objective: Science and tech...,2020-08-01,23
3,32411818,Report on computational assessment of Tumor In...,Assessment of tumor-infiltrating lymphocytes (...,2020-05-01,3
4,32410553,Application of Artificial Intelligence in Phar...,Background: Artificial inte...,2020-06-01,0


In [42]:
pm_df.dropna(inplace=True)

In [43]:
pm_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2986 entries, 0 to 3150
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   pmid       2986 non-null   int64         
 1   title      2986 non-null   object        
 2   text       2986 non-null   object        
 3   date       2986 non-null   datetime64[ns]
 4   citations  2986 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 140.0+ KB


#### Save Cleaned File

In [44]:
output_filename = '../data/pubmed_cleaned.csv'

In [45]:
pm_df.to_csv(output_filename, index=False)